In [2]:
import glob
import random
import base64
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from PIL import Image
from io import BytesIO
from IPython.display import HTML
import io
import re
import os
import sys
import pandas as pd
from p_tqdm import p_umap
from PIL import Image 
#displaying images in dataframe: https://stackoverflow.com/questions/46107348/how-to-display-image-stored-in-pandas-dataframe

# Compare input with results

In [86]:
preds_path= '/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/cleaned/interim_ddfull_cohort_output.csv'
input_path = '/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/cleaned/deduplicated_all.csv'

In [87]:
preds = pd.read_csv(preds_path)
df = pd.read_csv(input_path)

In [88]:
df

,Unnamed: 0,patient_id,exam_id,laterality,view,years_to_cancer,years_to_last_followup,file_path,split_group
0,0,91783139,2O11253,L,CC,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.81.1/1.2.840.113681.2229458196.18875.3506767646.84.1.dcm.png,test
1,1,91783139,2O11253,L,MLO,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.83.1/1.2.840.113681.2229458196.18875.3506767646.90.1.dcm.png,test
2,2,91783139,2O11253,R,MLO,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.82.1/1.2.840.113681.2229458196.18875.3506767646.88.1.dcm.png,test
3,3,91783139,2O11253,R,CC,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.80.1/1.2.840.113681.2229458196.18875.3506767646.86.1.dcm.png,test
4,4,58982377,2O10729,L,CC,100,4,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/58982377/2O10729/1.2.840.113681.2772716983.1445237983.4760.13698/1.2.840.113681.2772716983.1445237983.4760.13699.dcm.png,test
...,...,...,...,...,...,...,...,...,...
28282,29373,99886953,2O14530,R,CC,100,3,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/99886953/2O14530/1.2.840.113681.2229458432.1742.3681463029.72.1/1.2.840.113681.2229458432.1742.3681463029.74.1.dcm.png,test
28283,29374,99886953,2O14529,R,CC,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/99886953/2O14529/1.2.840.113681.2229458432.6383.3698311237.75.1/1.2.840.113681.2229458432.6383.3698311237.79.1.dcm.png,test
28284,29375,99886953,2O14529,L,MLO,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/99886953/2O14529/1.2.840.113681.2229458432.6383.3698311237.77.1/1.2.840.113681.2229458432.6383.3698311237.87.1.dcm.png,test
28285,29376,99886953,2O14529,L,CC,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/99886953/2O14529/1.2.840.113681.2229458432.6383.3698311237.76.1/1.2.840.113681.2229458432.6383.3698311237.85.1.dcm.png,test


In [89]:
df = df.drop(df.columns[0], axis=1) #drop uncessary column 

In [90]:
df['patient_id'] = df['patient_id'].astype(str)
df["exam_id"] = df['exam_id'].astype(str)
df['patient_exam_id'] = df['patient_id'] + '\t' + df['exam_id']
df.head(5)

,patient_id,exam_id,laterality,view,years_to_cancer,years_to_last_followup,file_path,split_group,patient_exam_id
0,91783139,2O11253,L,CC,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.81.1/1.2.840.113681.2229458196.18875.3506767646.84.1.dcm.png,test,91783139\t2O11253
1,91783139,2O11253,L,MLO,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.83.1/1.2.840.113681.2229458196.18875.3506767646.90.1.dcm.png,test,91783139\t2O11253
2,91783139,2O11253,R,MLO,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.82.1/1.2.840.113681.2229458196.18875.3506767646.88.1.dcm.png,test,91783139\t2O11253
3,91783139,2O11253,R,CC,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.80.1/1.2.840.113681.2229458196.18875.3506767646.86.1.dcm.png,test,91783139\t2O11253
4,58982377,2O10729,L,CC,100,4,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/58982377/2O10729/1.2.840.113681.2772716983.1445237983.4760.13698/1.2.840.113681.2772716983.1445237983.4760.13699.dcm.png,test,58982377\t2O10729


In [91]:
df['idx'] = df.groupby('patient_exam_id').cumcount()
#https://stackoverflow.com/questions/22798934/pandas-long-to-wide-reshape-by-two-variables
tmp = []
for var in ['laterality','view', 'file_path']:
    df['tmp_idx'] = var + '_' + df.idx.astype(str)
    tmp.append(df.pivot(index='patient_exam_id',columns='tmp_idx',values=var))

reshape = pd.concat(tmp,axis=1)

In [92]:
reshape.head(5)

tmp_idx,laterality_0,laterality_1,laterality_10,laterality_11,laterality_12,laterality_2,laterality_3,laterality_4,laterality_5,laterality_6,...,file_path_11,file_path_12,file_path_2,file_path_3,file_path_4,file_path_5,file_path_6,file_path_7,file_path_8,file_path_9
patient_exam_id,,,,,,,,,,,,,,,,,,,,,
10000462\t2O03320,R,L,NaN,NaN,NaN,L,R,NaN,NaN,NaN,...,NaN,NaN,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10000462/2O03320/1.2.826.0.1.3680043.8.829.2.2.2.1.4223.500.1/1.2.826.0.1.3680043.8.829.2.2.2.1.4223.500.1.2.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10000462/2O03320/1.2.826.0.1.3680043.8.829.2.2.2.1.4223.500.2/1.2.826.0.1.3680043.8.829.2.2.2.1.4223.500.2.3.dcm.png,NaN,NaN,NaN,NaN,NaN,NaN
10034806\t2O03318,R,L,NaN,NaN,NaN,L,R,R,L,R,...,NaN,NaN,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10034806/2O03318/1.2.840.113681.181348746.1450333476.5352.137948.1/1.2.840.113681.181348746.1450333476.5352.137949.1.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10034806/2O03318/1.2.840.113681.181348746.1450333476.5352.137898/1.2.840.113681.181348746.1450333476.5352.137899.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10034806/2O03318/1.2.840.113681.181348746.1450333476.5352.137912/1.2.840.113681.181348746.1450333476.5352.137913.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10034806/2O03318/1.2.840.113681.181348746.1450333476.5352.137926/1.2.840.113681.181348746.1450333476.5352.137927.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10034806/2O03318/1.2.840.113681.181348746.1450333476.5352.137920.1/1.2.840.113681.181348746.1450333476.5352.137921.1.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10034806/2O03318/1.2.840.113681.181348746.1450333476.5352.138024/1.2.840.113681.181348746.1450333476.5352.138025.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10034806/2O03318/1.2.840.113681.181348746.1450333476.5352.137940/1.2.840.113681.181348746.1450333476.5352.137941.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10034806/2O03318/1.2.840.113681.181348746.1450333476.5352.138038/1.2.840.113681.181348746.1450333476.5352.138039.dcm.png
10127072\t2O03649,R,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10127072\t2O03651,L,R,NaN,NaN,NaN,L,L,R,R,L,...,NaN,NaN,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10127072/2O03651/1.2.840.113681.171803414.1474965906.5152.15247.1/1.2.840.113681.171803414.1474965906.5152.15248.1.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10127072/2O03651/1.2.840.113681.171803414.1474965906.5152.15239/1.2.840.113681.171803414.1474965906.5152.15240.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10127072/2O03651/1.2.840.113681.171803414.1474965906.5152.15215.1/1.2.840.113681.171803414.1474965906.5152.15216.1.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10127072/2O03651/1.2.840.113681.171803414.1474965906.5152.15223/1.2.840.113681.171803414.1474965906.5152.15224.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10127072/2O03651/1.2.840.113681.171803414.1474965906.5152.15263.1/1.2.840.113681.171803414.1474965906.5152.15264.1.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10127072/2O03651/1.2.840.113681.171803414.1474965906.5152.15207/1.2.840.113681.171803414.1474965906.5152.15208.dcm.png,NaN,NaN
10127072\t2O03652,L,L,NaN,NaN,NaN,R,R,NaN,NaN,NaN,...,NaN,NaN,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10127072/2O03652/1.2.840.113681.2229432822.6432.360

In [93]:
print('number of patient_exam_id pairs in input table is {}.'.format(len(reshape)))
print('number of patient_exam_id pairs in results table is {}'.format(len(preds)))
#this is with the deduplicated input

number of patient_exam_id pairs in input table is 6262.
number of patient_exam_id pairs in results table is 5416


In [94]:
reshape = reshape.merge(preds, how="left", on="patient_exam_id", indicator=True)
reshape._merge.value_counts()  # all patients in CRDW represented in mrn_to_studyid map #all merged

both          5416
left_only      846
right_only       0
Name: _merge, dtype: int64

In [95]:
#see the exams that did not pass into the model
filtered = reshape[reshape._merge=="left_only"]
pd.set_option('display.max_colwidth', None)
filtered.head(10)

,patient_exam_id,laterality_0,laterality_1,laterality_10,laterality_11,laterality_12,laterality_2,laterality_3,laterality_4,laterality_5,...,file_path_6,file_path_7,file_path_8,file_path_9,1_year_risk,2_year_risk,3_year_risk,4_year_risk,5_year_risk,_merge
2,10127072\t2O03649,R,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
53,10851115\t2O03286,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
54,10861161\t2O03691,L,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
55,10861161\t2O03693,L,L,NaN,NaN,NaN,L,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
56,10861161\t2O03694,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
81,11082929\t2O03553,L,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
103,11578260\t2O03720,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
104,11578260\t2O03721,R,R,NaN,NaN,NaN,R,R,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
105,11578260\t2O03722,L,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
109,11634771\t2O17738,L,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [96]:
filtered.dropna(axis=1, how='all', inplace=True) #drop columns that are all NA
filtered.head(10)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,patient_exam_id,laterality_0,laterality_1,laterality_2,laterality_3,laterality_4,laterality_5,laterality_6,laterality_7,view_0,...,view_7,file_path_0,file_path_1,file_path_2,file_path_3,file_path_4,file_path_5,file_path_6,file_path_7,_merge
2,10127072\t2O03649,R,R,NaN,NaN,NaN,NaN,NaN,NaN,CC,...,NaN,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10127072/2O03649/1.2.840.113681.2229461946.970.3659176020.7.1/1.2.840.113681.2229461946.970.3659176020.13.1.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10127072/2O03649/1.2.840.113681.2229461946.970.3659176020.8.1/1.2.840.113681.2229461946.970.3659176020.19.1.dcm.png,NaN,NaN,NaN,NaN,NaN,NaN,left_only
53,10851115\t2O03286,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC,...,NaN,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10851115/2O03286/1.2.840.113681.2223034036.958.3401963797.5.1/1.2.840.113681.2223034036.958.3401963797.12.1.dcm.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
54,10861161\t2O03691,L,L,NaN,NaN,NaN,NaN,NaN,NaN,CC,...,NaN,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10861161/2O03691/1.2.840.113681.2229462448.969.3492230935.26.1/1.2.840.113681.2229462448.969.3492230935.30.1.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10861161/2O03691/1.2.840.113681.2229462448.969.3492230935.28.1/1.2.840.113681.2229462448.969.3492230935.33.1.dcm.png,NaN,NaN,NaN,NaN,NaN,NaN,left_only
55,10861161\t2O03693,L,L,L,NaN,NaN,NaN,NaN,NaN,CC,...,NaN,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10861161/2O03693/1.2.840.113681.2229462448.969.3492230935.14.1/1.2.840.113681.2229462448.969.3492230935.17.1.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10861161/2O03693/1.2.840.113681.2229462448.969.3492230935.3.1/1.2.840.113681.2229462448.969.3492230935.12.1.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10861161/2O03693/1.2.840.113681.2229462448.969.3492230935.4.1/1.2.840.113681.2229462448.969.3492230935.8.1.dcm.png,NaN,NaN,NaN,NaN,NaN,left_only
56,10861161\t2O03694,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC,...,NaN,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/10861161/2O03694/1.2.840.113681.2229462448.965.3494823826.33.1/1.2.840.113681.2229462448.965.3494823826.35.1.dcm.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
81,11082929\t2O03553,L,L,NaN,NaN,NaN,NaN,NaN,NaN,CC,...,NaN,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/11082929/2O03553/1.2.840.113681.2229461946.3092.3590919253.155.1/1.2.840.113681.2229461946.3092.3590919253.158.1.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/11082929/2O03553/1.2.840.113681.2229461946.3092.3590919253.156.1/1.2.840.113681.2229461946.3092.3590919253.160.1.dcm.png,NaN,NaN,NaN,NaN,NaN,NaN,left_only
103,11578260\t2O03720,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC,...,NaN,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/11578260/2O03720/2.16.840.1.113786.1.234.9933.750179255.244/2.16.840.1.113786.1.234.9933.750179255.245.dcm.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
104,11578260\t2O03721,R,R,R,R,NaN,NaN,NaN,NaN,CC,...,NaN,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/11578260/2O03721/1.2.840.113681.2229461946.975.3663323472.110.1/1.2.840.113681.2229461946.975.3663323472.113.1.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/11578260/2O03721/1.2.840.113681.2229461946.975.3663323472.111.1/1.2.840.113681.2229461946.975.3663323472.120.1.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/11578260/2O03721/1.2.840.113681.2229461946.975.3663323472.115.1/1.2.840.113681.2229461946.975.3663323472.118.1.dcm.png,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/11578260/2

In [107]:
filtered.columns.to_list()
filtered = filtered.filter(regex='^(?!f).*', axis=1) # remove columns starting with f

In [108]:
filtered.head(20)

,patient_exam_id,laterality_0,laterality_1,laterality_2,laterality_3,laterality_4,laterality_5,laterality_6,laterality_7,view_0,view_1,view_2,view_3,view_4,view_5,view_6,view_7,_merge
2,10127072\t2O03649,R,R,NaN,NaN,NaN,NaN,NaN,NaN,CC,CC,NaN,NaN,NaN,NaN,NaN,NaN,left_only
53,10851115\t2O03286,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
54,10861161\t2O03691,L,L,NaN,NaN,NaN,NaN,NaN,NaN,CC,CC,NaN,NaN,NaN,NaN,NaN,NaN,left_only
55,10861161\t2O03693,L,L,L,NaN,NaN,NaN,NaN,NaN,CC,CC,MLO,NaN,NaN,NaN,NaN,NaN,left_only
56,10861161\t2O03694,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
81,11082929\t2O03553,L,L,NaN,NaN,NaN,NaN,NaN,NaN,CC,MLO,NaN,NaN,NaN,NaN,NaN,NaN,left_only
103,11578260\t2O03720,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
104,11578260\t2O03721,R,R,R,R,NaN,NaN,NaN,NaN,CC,MLO,CC,MLO,NaN,NaN,NaN,NaN,left_only
105,11578260\t2O03722,L,R,NaN,NaN,NaN,NaN,NaN,NaN,MLO,MLO,NaN,NaN,NaN,NaN,NaN,NaN,left_only
109,11634771\t2O17738,L,L,NaN,NaN,NaN,NaN,NaN,NaN,MLO,CC,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [99]:
# exams filtered out do not have all four complimentary views

NameError: name 'end' is not defined

BRINGING THE FILTER HERE TO COUNT CASES AND CONTROLS INVOLVED

In [ ]:
metadata_path= "../data/chimec_mammo_retry.csv" 
metadata= pd.read_csv(metadata_path) #contains full image metadata information with image means
""""
This csv is result of a three-step process:
1. Metadata extraction from dicom images that pass cut (conventional views, FOR PRESENTATION)
2. Filtering out metadata for pngs that were not created
3. Generation of Image means and appending to metadata

"""
# pd.set_option("max_colwidth", None) #do not restrict column width


Columns (8,9,11,27,40,41,43,45,49,57,58,65,75,82,85,87,88,91,95,104,147,152,167,168,176,183,204,216,226,227,228,229,232,234,238,242,245,248,249,252,253,255,257,262,263,264,266,267,268,269,275,276,277,278,279,280,281,282,283,284,287,289,292,293,294,295,296,299,302,303) have mixed types.Specify dtype option on import or set low_memory=False.


'"\nThis csv is result of a three-step process:\n1. Metadata extraction from dicom images that pass cut (conventional views, FOR PRESENTATION)\n2. Filtering out metadata for pngs that were not created\n3. Generation of Image means and appending to metadata\n\n'

# Filteration Strategy:
Order:
1. Compression Images
2. Breast Implant Images
3. Unprocessed Images
4. Image mean outliers
5. Date filtering: if exam is after diagnosis 

For each filter:
1. create residual dataframe with png paths
2. export residual dataframe as csv
3. run thumbnail creation into named residual folder for inspection

In [ ]:
def print_summary(metadata):
    total_n = len(metadata) #total number of dicom files
    total_p = metadata.study_id.nunique() #total number of patients
    total_e = metadata.exam_id.nunique() #total number of exams
    total_CE = metadata[metadata.case==True].exam_id.nunique() #total number of case exams
    total_NE = metadata[metadata.case==False].exam_id.nunique() # total number of control exams
    total_cases = metadata[metadata.case==True].study_id.nunique() # total number of cases
    total_controls = metadata[metadata.case==False].study_id.nunique() # total number of controls
    return print("{}AVAILABLE/LOADED{} \n {} patients \n {} case exams for {} cases \n {} control exams for {} controls \n {} total files".format(
        "*"*5, "*"*5,
        total_p,
        total_CE, total_cases,
        total_NE, total_controls,
        total_n))

In [ ]:
print_summary(metadata)

*****AVAILABLE/LOADED***** 
 2030 patients 
 1607 case exams for 191 cases 
 10585 control exams for 1839 controls 
 54535 total files


In [ ]:
#filter for png path existence

def png_exist_filter(metadata):
    """
    takes in dataframe with png paths as argument
    filters out rows with non-exitent corresponding pngs
    returns filtered dataframe
    """
    total_n = len(metadata) #total number of dicom files
    total_p = metadata.study_id.nunique() #total number of patients
    total_e = metadata.exam_id.nunique() #total number of exams
    total_CE = metadata[metadata.case==True].exam_id.nunique() #total number of case exams
    total_NE = metadata[metadata.case==False].exam_id.nunique() # total number of control exams
    total_cases = metadata[metadata.case==True].study_id.nunique() # total number of cases
    total_controls = metadata[metadata.case==False].study_id.nunique() # total number of controls
    metadata["png_true"]= metadata.png_path.apply(lambda x: os.path.isfile(x))
    metadata = metadata[metadata.png_true == True]
    print(
        "will filter out {} files, {} case exams for {} unique cases and {} control exams for {} unique controls".format(
        total_n - len(metadata), total_CE - metadata[metadata.case==True].exam_id.nunique(), 
        total_cases - metadata[metadata.case==True].study_id.nunique(),
        total_NE - metadata[metadata.case==False].exam_id.nunique(),
        total_controls - metadata[metadata.case==False].study_id.nunique())
    )
    return metadata 

In [ ]:
metadata = png_exist_filter(metadata) #filtering data based on png existence 

will filter out 16344 files, 577 case exams for 19 unique cases and 3486 control exams for 187 unique controls


In [ ]:
print_summary(metadata)

*****AVAILABLE/LOADED***** 
 1824 patients 
 1030 case exams for 172 cases 
 7099 control exams for 1652 controls 
 38191 total files


In [ ]:

print("Commence filtering and exporting thumbnails of cut flows")

Commence filtering and exporting thumbnails of cut flows


In [ ]:
def compression_filter(metadata):
    """
    filters out compression images based on metadata labels, outputs csv of paths of filtered out images, and returns filtered metadata
    """
    total_n = len(metadata) #total number of dicom files
    total_p = metadata.study_id.nunique() #total number of patients
    total_e = metadata.exam_id.nunique() #total number of exams
    total_CE = metadata[metadata.case==True].exam_id.nunique() #total number of case exams
    total_NE = metadata[metadata.case==False].exam_id.nunique() # total number of control exams
    total_cases = metadata[metadata.case==True].study_id.nunique() # total number of cases
    total_controls = metadata[metadata.case==False].study_id.nunique() # total number of controls
    compression_mask = metadata.ViewCodeSequence.apply(lambda x: "ompression" in x) #search for string indicating compression in ViewCodeSequence key (which is an array of lists)
    lost = metadata[compression_mask]
    metadata = metadata[~compression_mask]
    print(
    "will filter out {} files, {} case exams for {} unique cases and {} control exams for {} unique controls in first compression filter".format(
    total_n - len(metadata), total_CE - metadata[metadata.case==True].exam_id.nunique(), 
    total_cases - metadata[metadata.case==True].study_id.nunique(),
    total_NE - metadata[metadata.case==False].exam_id.nunique(),
    total_controls - metadata[metadata.case==False].study_id.nunique())
    )
    #Query a series of lists for strings: https://stackoverflow.com/questions/41518920/python-pandas-how-to-query-if-a-list-type-column-contains-something
    #apply second compression filter
    total_n = len(metadata) #total number of dicom files
    total_p = metadata.study_id.nunique() #total number of patients
    total_e = metadata.exam_id.nunique() #total number of exams
    total_CE = metadata[metadata.case==True].exam_id.nunique() #total number of case exams
    total_NE = metadata[metadata.case==False].exam_id.nunique() # total number of control exams
    total_cases = metadata[metadata.case==True].study_id.nunique() # total number of cases
    total_controls = metadata[metadata.case==False].study_id.nunique() # total number of controls
    pick = ["MCC", "SMLO", "SCC", "MCCID", "MMLO"] #pick: identifiers of parameter of interest
    compression_mask2= metadata.ProtocolName.str.contains('|'.join(pick)) == True 
    lost2= metadata[compression_mask2] #part of dataframe to be filtered out
    metadata = metadata[~compression_mask2] 
    print(
    "will filter out {} files, {} case exams for {} unique cases and {} control exams for {} unique controls in second compression filter".format(
    total_n - len(metadata), total_CE - metadata[metadata.case==True].exam_id.nunique(), 
    total_cases - metadata[metadata.case==True].study_id.nunique(),
    total_NE - metadata[metadata.case==False].exam_id.nunique(),
    total_controls - metadata[metadata.case==False].study_id.nunique())
    )
    #append paths to compression images to an array and export that into a csv file
    compression_paths1 = lost.png_path
    compression_paths2 = lost2.png_path
    all_compression_paths = compression_paths1.append(compression_paths2)
    all_compression_paths.to_csv("/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/compression_paths.csv")
    return metadata

    


In [ ]:
metadata= compression_filter(metadata)


will filter out 1442 files, 22 case exams for 0 unique cases and 199 control exams for 3 unique controls in first compression filter
will filter out 23 files, 1 case exams for 0 unique cases and 6 control exams for 0 unique controls in second compression filter


In [ ]:
print_summary(metadata)


*****AVAILABLE/LOADED***** 
 1821 patients 
 1007 case exams for 172 cases 
 6894 control exams for 1649 controls 
 36726 total files


In [ ]:
## convert compression pngs to thumbnails and keep in a "compression" folder
import ntpath
ntpath.basename("a/b/c")
size = (100, 100)
#extract the last file name in path: https://stackoverflow.com/questions/8384737/extract-file-name-from-path-no-matter-what-the-os-path-format

png_dir = "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs"
thumb_dir = "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/compression_images"
series_metadata_path = "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/compression_paths.csv"

def create_thumbnail_path (png_path, png_dir, thumb_dir):
    """Converts a DICOM path to a PNG path by replacing dicom_dir with png_dir in the path.

    Arguments:
        png_dir(str): Path to a directory containing PNG images
        thumb_dir(str): Path to the directory where thumbnail versions of PNGS
        will be stored

    Returns:
        path to thumbnails
    """
    head, tail = ntpath.split(png_path)
    png_name = tail 
    thumb_path = os.path.join(thumb_dir, tail)
    return thumb_path 

paths = pd.read_csv(series_metadata_path)
df = pd.DataFrame(paths)
png_paths= df.png_path.tolist()
print(f"Converting {len(png_paths)} PNGs to thumbnails")

# for image in png_paths:
#     try:
#         im = Image.open(image)
#     except IOError as e:
#         #report error and then skip to the next argument
#         print("Problem opening", image, ":", e)
#         continue
#     im.resize(size)
#     path = create_thumbnail_path(image, png_dir, thumb_dir)
#     os.makedirs(os.path.dirname(path), exist_ok=True)
#     im.save(path)

print ("completed thumbnail directory of {} compressed images". format(len(png_paths)))

Converting 1465 PNGs to thumbnails
completed thumbnail directory of 1465 compressed images


In [ ]:
def implant_filter(metadata):
    """
    filters out breast implant images based on metadata labels, outputs csv of paths of filtered out images, and returns filtered metadata
    """
    total_n = len(metadata) #total number of dicom files
    total_p = metadata.study_id.nunique() #total number of patients
    total_e = metadata.exam_id.nunique() #total number of exams
    total_CE = metadata[metadata.case==True].exam_id.nunique() #total number of case exams
    total_NE = metadata[metadata.case==False].exam_id.nunique() # total number of control exams
    total_cases = metadata[metadata.case==True].study_id.nunique() # total number of cases
    total_controls = metadata[metadata.case==False].study_id.nunique() # total number of controls
    implant_mask1 = metadata["BreastImplantPresent"] == "YES" #not always specified as "NO" if not present
    lost1 = metadata[implant_mask1] 
    metadata = metadata[~implant_mask1] 
    print(
    "will filter out {} files, {} case exams for {} unique cases and {} control exams for {} unique controls in breast implant filter".format(
    total_n - len(metadata), total_CE - metadata[metadata.case==True].exam_id.nunique(), 
    total_cases - metadata[metadata.case==True].study_id.nunique(),
    total_NE - metadata[metadata.case==False].exam_id.nunique(),
    total_controls - metadata[metadata.case==False].study_id.nunique())
    )
    total_n = len(metadata) #total number of dicom files
    total_p = metadata.study_id.nunique() #total number of patients
    total_e = metadata.exam_id.nunique() #total number of exams
    total_CE = metadata[metadata.case==True].exam_id.nunique() #total number of case exams
    total_NE = metadata[metadata.case==False].exam_id.nunique() # total number of control exams
    total_cases = metadata[metadata.case==True].study_id.nunique() # total number of cases
    total_controls = metadata[metadata.case==False].study_id.nunique() # total number of controls
    pick = ["MLOID", "CCID"] #pick: identifiers of parameter of interest
    implant_mask2= metadata.ProtocolName.str.contains('|'.join(pick)) == True 
    lost2= metadata[implant_mask2] 
    metadata = metadata[~implant_mask2] 
    print(
    "will filter out {} files, {} case exams for {} unique cases and {} control exams for {} unique controls in second breast implant filter".format(
    total_n - len(metadata), total_CE - metadata[metadata.case==True].exam_id.nunique(), 
    total_cases - metadata[metadata.case==True].study_id.nunique(),
    total_NE - metadata[metadata.case==False].exam_id.nunique(),
    total_controls - metadata[metadata.case==False].study_id.nunique())
    )
    #append paths to implant images to an array and export that into a csv file
    implant_paths1 = lost1.png_path
    implant_paths2 = lost2.png_path
    all_implant_paths = implant_paths1.append(implant_paths2)
    all_implant_paths.to_csv("/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/implant_paths.csv")
    return metadata

In [ ]:
metadata=implant_filter(metadata)

will filter out 547 files, 6 case exams for 2 unique cases and 86 control exams for 23 unique controls in breast implant filter
will filter out 8 files, 1 case exams for 0 unique cases and 2 control exams for 2 unique controls in second breast implant filter


In [ ]:
print_summary(metadata)

*****AVAILABLE/LOADED***** 
 1794 patients 
 1000 case exams for 170 cases 
 6806 control exams for 1624 controls 
 36171 total files


In [ ]:
def implant_filter(metadata):
    """
    filters out breast implant images based on time of exam relative to diagnosis
    , outputs csv of paths of filtered out images, and returns filtered metadata
    """
    total_n = len(metadata) #total number of dicom files
    total_p = metadata.study_id.nunique() #total number of patients
    total_e = metadata.exam_id.nunique() #total number of exams
    total_CE = metadata[metadata.case==True].exam_id.nunique() #total number of case exams
    total_NE = metadata[metadata.case==False].exam_id.nunique() # total number of control exams
    total_cases = metadata[metadata.case==True].study_id.nunique() # total number of cases
    total_controls = metadata[metadata.case==False].study_id.nunique() # total number of controls
    negative_filter = metadata.dx_years_after_exam < 0.1 #0.1 years (36.5 days as minimum cut-off time from exam to diagnosis)
    lost1 = metadata[negative_filter] 
    metadata = metadata[~negative_filter] 
    print(
    "will filter out {} files, {} case exams for {} unique cases and {} control exams for {} unique controls in examination time filter".format(
    total_n - len(metadata), total_CE - metadata[metadata.case==True].exam_id.nunique(), 
    total_cases - metadata[metadata.case==True].study_id.nunique(),
    total_NE - metadata[metadata.case==False].exam_id.nunique(),
    total_controls - metadata[metadata.case==False].study_id.nunique())
    )
    
    #append paths to implant images to an array and export that into a csv file
    time_paths1 = lost1.png_path
    time_paths1.to_csv("/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/time_paths.csv")
    return metadata

In [ ]:
metadata = implant_filter(metadata)

will filter out 2617 files, 623 case exams for 26 unique cases and 0 control exams for 0 unique controls in examination time filter


In [ ]:
print_summary(metadata)

*****AVAILABLE/LOADED***** 
 1768 patients 
 377 case exams for 144 cases 
 6806 control exams for 1624 controls 
 33554 total files


Next, look at image mean outliers

In [ ]:
#CONTINUE CLEANING FROM HERE 



# #generate image means
# metadata["image_means"]= metadata['png_path'].apply(lambda x: np.mean(np.asarray(Image.open(x))))

# #save metadata with image means and then reload

# metadata.to_csv("../data/chimec_filtered_wmeans.csv" )
metadata_path= "../data/chimec_filtered_wmeans.csv"
metadata= pd.read_csv(metadata_path)


Columns (5,6,9,10,12,28,41,42,44,50,58,59,66,76,86,88,89,92,96,105,148,159,160,177,184,192,194,195,196,197,200,201,202,204,206,207,226,227,228,230,233,235,239,243,246,249,250,253,254,256,258,263,264,265,267,268,269,270,276,277,278,279,280,281,282,283,284,285,288,290,293,294,295,296,297,303,304) have mixed types.Specify dtype option on import or set low_memory=False.


In [ ]:
foo_mask = metadata.ViewCodeSequence.apply(lambda x: "tissue" in x) #search for string indicating tissue(maybe surgical specimen) in ViewCodeSequence key (which is an array of lists)
foodf = metadata[foo_mask]
#Query a series of lists for strings: https://stackoverflow.com/questions/41518920/python-pandas-how-to-query-if-a-list-type-column-contains-something

In [ ]:
df = metadata 
print("image means span from minimum of {} to maximum of {}, with an mean of {} and sd of {}".format(
    df.image_means.min(), df.image_means.max(), df.image_means.mean(), df.image_means.std()
    )
)# histogram of image means
x = df.image_means
n, bins, patches = plt.hist(x, 50, facecolor='g')

plt.xlabel('Image mean intensity')
plt.ylabel('Frequency')
plt.title('Histogram of Image Means')
# plt.text(images.image_means.mean(), images.image_means.std(), r'$\mu=,\ \sigma=15$')
plt.xlim(df.image_means.min() - 50 , df.image_means.max() + 50)
#plt.ylim(0, 0.03)
plt.grid(True)
plt.show()

image means span from minimum of 931.3939348660982 to maximum of 41474.59936786413, with an mean of 9121.599260177441 and sd of 4596.756837663727


Image means still right skewed. Do further metadata based cutting before cutting based on image means?

2SD cut?
3SD cut? - look at metadata and images for each of the cuts

Filter based on Image Type Attribute
https://dicom.innolitics.com/ciods/rt-dose/general-image/00080008



In [ ]:
total_n = len(metadata)
total_p = metadata.study_id.nunique() #total number of patients
total_e = metadata.exam_id.nunique() #total number of exams
total_CE = metadata[metadata.case==True].exam_id.nunique() #total number of case exams
total_NE = metadata[metadata.case==False].exam_id.nunique() # total number of control exams
total_cases = metadata[metadata.case==True].study_id.nunique() # total number of cases
total_controls = metadata[metadata.case==False].study_id.nunique() # total number of controls
print(" {}AVAILABLE/LOADED{}".format("*" * 18,"*" * 18 ))
print(
    "{} PNGS/paths \n {} patients \n {} CASES and {} CONTROLS \n {} case exams and {} control exams".format(
        total_n, total_p, total_cases, total_controls, total_CE, total_NE
    )
)

 ******************AVAILABLE/LOADED******************
36171 PNGS/paths 
 1794 patients 
 170 CASES and 1624 CONTROLS 
 1000 case exams and 6806 control exams


In [ ]:
raw_mask = metadata.ImageType.apply(lambda x: "ORIGINAL" in x) #search for string indicating ? unprocessed in ImageType key (which is an array of lists)
lost = metadata[raw_mask]
metadata = metadata[~raw_mask]
#Query a series of lists for strings: https://stackoverflow.com/questions/41518920/python-pandas-how-to-query-if-a-list-type-column-contains-something

In [ ]:
#place png paths of ORIGINAL images in a list and run thumbnail creation
raw_original_paths = lost.png_path
raw_original_paths.to_csv("/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/raw_original_paths.csv")

In [ ]:
# ## convert raw? original pngs to thumbnails and keep in an specified folder
# import ntpath
# ntpath.basename("a/b/c")
# size = (100, 100)
# #extract the last file name in path: https://stackoverflow.com/questions/8384737/extract-file-name-from-path-no-matter-what-the-os-path-format

# png_dir = "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs"
# thumb_dir = "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/raw_original_images"
# series_metadata_path = "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/raw_original_paths.csv"

# def create_thumbnail_path (png_path, png_dir, thumb_dir):
#     """Converts a DICOM path to a PNG path by replacing dicom_dir with png_dir in the path.

#     Arguments:
#         png_dir(str): Path to a directory containing PNG images
#         thumb_dir(str): Path to the directory where thumbnail versions of PNGS
#         will be stored

#     Returns:
#         path to thumbnails
#     """
#     head, tail = ntpath.split(png_path)
#     png_name = tail 
#     thumb_path = os.path.join(thumb_dir, tail)
#     return thumb_path 

# paths = pd.read_csv(series_metadata_path)
# df = pd.DataFrame(paths)
# png_paths= df.png_path.tolist()
# print(f"Converting {len(png_paths)} PNGs to thumbnails")

# for image in png_paths:
#     try:
#         im = Image.open(image)
#     except IOError as e:
#         #report error and then skip to the next argument
#         print("Problem opening", image, ":", e)
#         continue
#     im.resize(size)
#     path = create_thumbnail_path(image, png_dir, thumb_dir)
#     os.makedirs(os.path.dirname(path), exist_ok=True)
#     im.save(path)

# print ("completed thumbnail directory of {} raw_original images". format(len(png_paths)))

In [ ]:
metadata.ImageType.value_counts()

['DERIVED', 'SECONDARY']               22432
['DERIVED', 'PRIMARY']                  9361
DERIVED/PRIMARY                          150
['DERIVED', 'PRIMARY', '', 'RIGHT']      131
['DERIVED', 'PRIMARY', '', 'LEFT']       113
['DERIVED', 'SECONDARY', '', '']          10
['DIRIVED', 'PRIMARY']                    10
DERIVED|PRIMARY                            9
Name: ImageType, dtype: int64

In [ ]:
print(
    "filtered out {} pngs, {} CASE exams for {} unique cases and {} CONTROL exams for {} unique controls based on png creation".format(
        total_n - len(metadata), total_CE - metadata[metadata.case==True].exam_id.nunique(), 
        total_cases - metadata[metadata.case==True].study_id.nunique(),
        total_NE - metadata[metadata.case==False].exam_id.nunique(),
        total_controls - metadata[metadata.case==False].study_id.nunique()
    )
)

filtered out 3955 pngs, 103 CASE exams for 7 unique cases and 768 CONTROL exams for 48 unique controls based on png creation


(examine image means first)

In [ ]:
df = metadata 
print("image means span from minimum of {} to maximum of {}, with an mean of {} and sd of {}".format(
    df.image_means.min(), df.image_means.max(), df.image_means.mean(), df.image_means.std()
    )
)# histogram of image means
x = df.image_means
n, bins, patches = plt.hist(x, 50, facecolor='g')

plt.xlabel('Image mean intensity')
plt.ylabel('Frequency')
plt.title('Histogram of Image Means')
# plt.text(images.image_means.mean(), images.image_means.std(), r'$\mu=,\ \sigma=15$')
plt.xlim(df.image_means.min() - 50 , df.image_means.max() + 50)
#plt.ylim(0, 0.03)
plt.grid(True)
plt.show()

image means span from minimum of 931.3939348660982 to maximum of 37580.88660700872, with an mean of 8142.291512817822 and sd of 3594.0406295212983


Curve looks more normally distributed than before, SD reduced and mean image intensity reduced
Now apply Image mean cut from 3SD, check pngs


In [ ]:
df = metadata 
mean_limit= df.image_means.mean() + (2.5 * df.image_means.std())
mean_cut = df.image_means <= mean_limit
#applying image filtering
total_n = len(df.index)
df = df[mean_cut]
print(
    "will filter out {} out of {} pngs past mean cut".format(
        total_n - len(df.index), total_n
    )
)


will filter out 666 out of 32216 pngs past mean cut


In [ ]:
lost= metadata[metadata.image_means >= mean_limit]
metadata = df

mostly compression, implant and magnified images in first batch of image mean filter (90% or more)

In [ ]:
#place png paths of ORIGINAL images in a list and run thumbnail creation
above_mean_paths = lost.png_path
above_mean_paths.to_csv("/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/above_mean_25.csv")

In [ ]:
# ## convert above_mean pngs to thumbnails and keep in an specified folder
# import ntpath
# ntpath.basename("a/b/c")
# size = (100, 100)
# #extract the last file name in path: https://stackoverflow.com/questions/8384737/extract-file-name-from-path-no-matter-what-the-os-path-format

# png_dir = "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs"
# thumb_dir = "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/above_mean_images"
# series_metadata_path = "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/above_mean_25.csv"

# def create_thumbnail_path (png_path, png_dir, thumb_dir):
#     """Converts a DICOM path to a PNG path by replacing dicom_dir with png_dir in the path.

#     Arguments:
#         png_dir(str): Path to a directory containing PNG images
#         thumb_dir(str): Path to the directory where thumbnail versions of PNGS
#         will be stored

#     Returns:
#         path to thumbnails
#     """
#     head, tail = ntpath.split(png_path)
#     png_name = tail 
#     thumb_path = os.path.join(thumb_dir, tail)
#     return thumb_path 

# paths = pd.read_csv(series_metadata_path)
# df = pd.DataFrame(paths)
# png_paths= df.png_path.tolist()
# print(f"Converting {len(png_paths)} PNGs to thumbnails")

# for image in png_paths:
#     try:
#         im = Image.open(image)
#     except IOError as e:
#         #report error and then skip to the next argument
#         print("Problem opening", image, ":", e)
#         continue
#     im.resize(size)
#     path = create_thumbnail_path(image, png_dir, thumb_dir)
#     os.makedirs(os.path.dirname(path), exist_ok=True)
#     im.save(path)

# print ("completed thumbnail directory of {} above_mean25 images". format(len(png_paths)))

In [ ]:
#plot image means on a histogram to see distribution
df = metadata 
print("image means span from minimum of {} to maximum of {}, with an mean of {} and sd of {}".format(
    df.image_means.min(), df.image_means.max(), df.image_means.mean(), df.image_means.std()
    )
)# histogram of image means
x = df.image_means
n, bins, patches = plt.hist(x, 50, facecolor='g')

plt.xlabel('Image mean intensity')
plt.ylabel('Frequency')
plt.title('Histogram of Image Means')
# plt.text(images.image_means.mean(), images.image_means.std(), r'$\mu=,\ \sigma=15$')
plt.xlim(df.image_means.min() - 50 , df.image_means.max() + 50)
#plt.ylim(0, 0.03)
plt.grid(True)
plt.show()

image means span from minimum of 931.3939348660982 to maximum of 17117.114915187543, with an mean of 7858.079498496506 and sd of 3005.1743887105044


mean histogram looks more normally distributed but may still need to shave off some more
move to annotation filter

In [ ]:
#filter out images with burned-in annotation
annotation_mask= metadata.BurnedInAnnotation == "YES"
lost = metadata[annotation_mask]
metadata= metadata[~annotation_mask]

CUT based on follow_up time values (if negative)

In [ ]:
#filter out metadata with years after diagnosis as a negative value (had exam after date of diagnosis)
negative_filter = metadata.dx_years_after_exam < 0 
metadata[metadata.dx_years_after_exam < 0].case.value_counts() #confirms this only drops out cases
lost = metadata[negative_filter]
metadata = metadata[~negative_filter]

PREPARE FIILE TO BE PASSED INTO FIRST ITERATION OF MIRAI

In [ ]:
mirai_input = metadata
df = mirai_input

In [ ]:
columns_to_keep= ["study_id", "exam_id",
"ImageLaterality", "ViewPosition", 'case','dx_years_after_exam','follow_up_years', 'png_path']

df = df[columns_to_keep]

In [ ]:
key_dict= {"study_id": "patient_id", "exam_id": "exam_id", "ImageLaterality": "laterality", "ViewPosition": "view", "png_path":"file_path", "dx_years_after_exam": 
"years_to_cancer" , "follow_up_years": "years_to_last_followup"}
df= df.rename(mapper= key_dict, axis=1)
df.head(10)

,patient_id,exam_id,laterality,view,case,years_to_cancer,years_to_last_followup,file_path
0,91783139,2O11253,L,CC,False,NaN,2.241096,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.81.1/1.2.840.113681.2229458196.18875.3506767646.84.1.dcm.png
1,91783139,2O11253,L,MLO,False,NaN,2.241096,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.83.1/1.2.840.113681.2229458196.18875.3506767646.90.1.dcm.png
2,91783139,2O11253,R,MLO,False,NaN,2.241096,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.82.1/1.2.840.113681.2229458196.18875.3506767646.88.1.dcm.png
3,91783139,2O11253,R,CC,False,NaN,2.241096,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.80.1/1.2.840.113681.2229458196.18875.3506767646.86.1.dcm.png
4,58982377,2O10729,L,CC,False,NaN,4.049315,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/58982377/2O10729/1.2.840.113681.2772716983.1445237983.4760.13698/1.2.840.113681.2772716983.1445237983.4760.13699.dcm.png
6,58982377,2O10729,R,MLO,False,NaN,4.049315,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/58982377/2O10729/1.2.840.113681.2772716983.1445237983.4760.13662/1.2.840.113681.2772716983.1445237983.4760.13663.dcm.png
7,58982377,2O10729,R,CC,False,NaN,4.049315,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/58982377/2O10729/1.2.840.113681.2772716983.1445237983.4760.13606/1.2.840.113681.2772716983.1445237983.4760.13607.dcm.png
8,58982377,2O10729,R,CC,False,NaN,4.049315,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/58982377/2O10729/1.2.840.113681.2772716983.1445237983.4760.13620/1.2.840.113681.2772716983.1445237983.4760.13621.dcm.png
9,58982377,2O10729,L,MLO,False,NaN,4.049315,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/58982377/2O10729/1.2.840.113681.2772716983.1445237983.4760.13676/1.2.840.113681.2772716983.1445237983.4760.13677.dcm.png
10,58982377,2O10729,L,MLO,False,NaN,4.049315,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/58982377/2O10729/1.2.840.113681.2772716983.1445237983.4760.13742/1.2.840.113681.2772716983.1445237983.4760.13743.dcm.png


In [ ]:
df.loc[df["case"]== False, 'years_to_cancer'] = 100
#set years_to_cancer as 100 for controls
df['years_to_cancer']= df['years_to_cancer'].astype(int)
df.loc[df['case']==True, 'years_to_last_followup']= df['years_to_cancer']
# replace year_to_last_followup from NaN to years_to_cancer for cases
# & set year_to_cancer as integer type data 
df['years_to_last_followup'] = df['years_to_last_followup'].astype(int)
df['split_group']= 'test' #making split_group column for validation 

In [ ]:
df = df[['patient_id', 'exam_id', 'laterality', 'view', 'years_to_cancer', 'years_to_last_followup', 'file_path', 'split_group']]
df.to_csv(path_or_buf= "../data/cleaned/mirai_full_cohort.csv")

DIVIDE THE FULL COHORT INTO AGE BINS AND RUN ITERATIONS
1. Make dataframe of full cohort info to be fed into Mirai
2. Merge with mrn to study id sheet
3. Merge using mrn to dr_pats
4. Exclude subjects without a valid age
5. Make age bins
6. create separate dataframes using age bins
7. Run parallel script that finds predictions based on age groups

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/data/cleaned/mirai_full_cohort.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/data/cleaned/mirai_full_cohort.csv'

In [ ]:
data_path = "/gpfs/data/phs/groups/Projects/Huo_projects/SPORE/ojomoleye/data/CRDW_Registry_IndexDx_2020_Jul23.csv"
data = pd.read_csv(data_path)

mrn_to_study_id = pd.read_csv(
    "/gpfs/data/phs/groups/Projects/Huo_projects/SPORE/ojomoleye/data/mrn_to_study_id.csv",
    names=["mrn", "study_id"],
)


downloaded_images = pd.read_pickle(
    "/gpfs/data/phs/groups/Projects/Huo_projects/SPORE/ojomoleye/data/downloaded_imaging_metadata.pkl"
)


cases_and_controls = pd.read_csv(
    "/gpfs/data/phs/groups/Projects/Huo_projects/SPORE/ojomoleye/data/dr_7934_pats.txt",
    sep="|",
)

spore_registration = pd.read_csv(
    "/gpfs/data/phs/groups/Projects/Huo_projects/SPORE/ojomoleye/data/SPORERegistrationDat_DATA_2021-06-21_0927.csv"
)

Columns (0,16,17,43,46,50,51,82,108,110,111,135,136,138,140,156,157,158,159,161,176,178,179,180,182,183,184,185,217,221,225,226,229,230,232,233,234,235,236) have mixed types.Specify dtype option on import or set low_memory=False.
Columns (9,13,14,42,47,48,53,58,62,63,65,66,67,69,71,75,111,121,122,123,125,164,182,184,187,189,197,213,228,233,238,240,245,247,252,275,277,279,335,362,368,379,389,399,409,421,452,517,542,567,578,595,605,622,632,649,659,686,713,903,920,930,957,984,1011,1038,1228,1255,1282,1309,1336,1363,1390,1553,1579,1605,1631,1657) have mixed types.Specify dtype option on import or set low_memory=False.


In [ ]:
# study id and mrn both integer type in mrn_to_study_id
data = df
data["study_id"] = data["patient_id"]
data = data.merge(mrn_to_study_id, how="left", on="study_id", indicator=True)
data._merge.value_counts()  # all patients in CRDW represented in mrn_to_studyid map #all merged

both          29592
left_only         0
right_only        0
Name: _merge, dtype: int64

In [ ]:
data.drop(columns="_merge", inplace=True) #drop merge validation column
data.head()

,patient_id,exam_id,laterality,view,years_to_cancer,years_to_last_followup,file_path,split_group,study_id,mrn
0,91783139,2O11253,L,CC,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.81.1/1.2.840.113681.2229458196.18875.3506767646.84.1.dcm.png,test,91783139,1204041
1,91783139,2O11253,L,MLO,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.83.1/1.2.840.113681.2229458196.18875.3506767646.90.1.dcm.png,test,91783139,1204041
2,91783139,2O11253,R,MLO,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.82.1/1.2.840.113681.2229458196.18875.3506767646.88.1.dcm.png,test,91783139,1204041
3,91783139,2O11253,R,CC,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.80.1/1.2.840.113681.2229458196.18875.3506767646.86.1.dcm.png,test,91783139,1204041
4,58982377,2O10729,L,CC,100,4,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/58982377/2O10729/1.2.840.113681.2772716983.1445237983.4760.13698/1.2.840.113681.2772716983.1445237983.4760.13699.dcm.png,test,58982377,3488885


In [ ]:
#merge with dr_pats using mrn
col_2_keep= ['MRN', 'DATE_OF_BIRTH']
pt_age = cases_and_controls[col_2_keep]
pt_age.columns = map(str.lower, pt_age.columns)
pt_age.head()

,mrn,date_of_birth
0,8208,1976-03-17 00:00:00.000
1,75716,1933-01-24 00:00:00.000
2,83994,1933-06-16 00:00:00.000
3,129163,1935-06-03 00:00:00.000
4,139661,1935-11-01 00:00:00.000


In [ ]:
data = data.merge(pt_age, how="left", on="mrn", indicator=True)
data._merge.value_counts()  # all patients in CRDW represented in mrn_to_studyid map #all merged

both          29592
left_only         0
right_only        0
Name: _merge, dtype: int64

In [ ]:
data.drop(columns="_merge", inplace=True) #drop merge validation column
data.head()

,patient_id,exam_id,laterality,view,years_to_cancer,years_to_last_followup,file_path,split_group,study_id,mrn,date_of_birth
0,91783139,2O11253,L,CC,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.81.1/1.2.840.113681.2229458196.18875.3506767646.84.1.dcm.png,test,91783139,1204041,1964-12-17 00:00:00.000
1,91783139,2O11253,L,MLO,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.83.1/1.2.840.113681.2229458196.18875.3506767646.90.1.dcm.png,test,91783139,1204041,1964-12-17 00:00:00.000
2,91783139,2O11253,R,MLO,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.82.1/1.2.840.113681.2229458196.18875.3506767646.88.1.dcm.png,test,91783139,1204041,1964-12-17 00:00:00.000
3,91783139,2O11253,R,CC,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.80.1/1.2.840.113681.2229458196.18875.3506767646.86.1.dcm.png,test,91783139,1204041,1964-12-17 00:00:00.000
4,58982377,2O10729,L,CC,100,4,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/58982377/2O10729/1.2.840.113681.2772716983.1445237983.4760.13698/1.2.840.113681.2772716983.1445237983.4760.13699.dcm.png,test,58982377,3488885,1957-11-28 00:00:00.000


In [3]:
# use downloaded mammogram pkl file to extract date of exam and merge based on exam_id
downloaded_images = pd.read_pickle(
    "/gpfs/data/phs/groups/Projects/Huo_projects/SPORE/ojomoleye/data/downloaded_imaging_metadata.pkl"
)
foo = pd.DataFrame(downloaded_images)
foo = foo[foo["Modality"].str.contains("MG")]
foo = foo[~pd.isnull(foo.exam_id)]


,Modality,Study DateTime,StudyDescription,exam_id,study_id,DNA_available,time_since_exam,case,date_dx,dx_time_after_exam,dx_years_after_exam,date_of_last_contact,follow_up_time,follow_up_years,export_pending
0,MG,2012-03-09 09:37:27,"MAM DIGITAL W/CAD, RT",NaN,14974962,False,3367 days 07:59:16.929826,True,2011-10-27,-135 days +14:22:33,-0.369863,NaT,NaT,NaN,False
1,MG,2012-04-19 11:26:32,"MAM DIGITAL W/CAD, RT",NaN,14974962,False,3326 days 06:10:11.929826,True,2011-10-27,-176 days +12:33:28,-0.482192,NaT,NaT,NaN,False
2,MG,2014-02-05 10:45:36,"MAM DIGITAL W/CAD, LT",NaN,14974962,False,2669 days 06:51:07.929826,True,2011-10-27,-833 days +13:14:24,-2.282192,NaT,NaT,NaN,False
3,MG,2015-02-11 10:20:25,"MAM DIGITAL W/CAD, LT",NaN,14974962,False,2298 days 07:16:18.929826,True,2011-10-27,-1204 days +13:39:35,-3.298630,NaT,NaT,NaN,False
4,MG,2012-12-06 13:59:17,"MAM DIGITAL W/CAD, LT",NaN,14974962,False,3095 days 03:37:26.929826,True,2011-10-27,-407 days +10:00:43,-1.115068,NaT,NaT,NaN,False


In [ ]:
foo = foo[["Study DateTime", "exam_id", "study_id"]]
foo.head()

,Study DateTime,exam_id,study_id
12,2015-10-22 07:47:21,2O10729,58982377
13,2016-01-05 07:58:04,2O10730,58982377
17,2012-02-15 09:40:59,2O11253,91783139
18,2012-03-07 09:45:17,2O11254,91783139
21,2012-03-19 14:53:47,2O11255,91783139


In [ ]:
new_df = pd.merge(data, foo,  how='left', on=['study_id','exam_id'], indicator=True) #full merging occured
new_df._merge.value_counts()


both          32900
left_only         0
right_only        0
Name: _merge, dtype: int64

In [ ]:
new_df.drop(columns="_merge", inplace=True) #drop merge validation column
new_df.columns = map(str.lower, new_df.columns)
new_df["study_datetime"]= new_df["study datetime"] 
new_df.drop(columns="study datetime", inplace=True)

In [ ]:
new_df.study_datetime.dtype

dtype('<M8[ns]')

In [ ]:
new_df["study_datetime"] = pd.to_datetime(new_df["study_datetime"])
new_df["date_of_birth"] = pd.to_datetime(new_df["date_of_birth"])

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date

In [ ]:
#generate age in days and age in years
new_df["age_in_days"]= new_df["study_datetime"] - new_df["date_of_birth"] 
new_df["age_in_years"] = new_df["age_in_days"] / np.timedelta64(1,"Y")
new_df.head()

,patient_id,exam_id,laterality,view,years_to_cancer,years_to_last_followup,file_path,split_group,study_id,mrn,date_of_birth,study_datetime,age_in_days,age_in_years
0,91783139,2O11253,L,CC,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.81.1/1.2.840.113681.2229458196.18875.3506767646.84.1.dcm.png,test,91783139,1204041,1964-12-17,2012-02-15 09:40:59,17226 days 09:40:59,47.164291
1,91783139,2O11253,L,MLO,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.83.1/1.2.840.113681.2229458196.18875.3506767646.90.1.dcm.png,test,91783139,1204041,1964-12-17,2012-02-15 09:40:59,17226 days 09:40:59,47.164291
2,91783139,2O11253,R,MLO,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.82.1/1.2.840.113681.2229458196.18875.3506767646.88.1.dcm.png,test,91783139,1204041,1964-12-17,2012-02-15 09:40:59,17226 days 09:40:59,47.164291
3,91783139,2O11253,R,CC,100,2,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/91783139/2O11253/1.2.840.113681.2229458196.18875.3506767646.80.1/1.2.840.113681.2229458196.18875.3506767646.86.1.dcm.png,test,91783139,1204041,1964-12-17,2012-02-15 09:40:59,17226 days 09:40:59,47.164291
4,58982377,2O10729,L,CC,100,4,/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_pngs/58982377/2O10729/1.2.840.113681.2772716983.1445237983.4760.13698/1.2.840.113681.2772716983.1445237983.4760.13699.dcm.png,test,58982377,3488885,1957-11-28,2015-10-22 07:47:21,21147 days 07:47:21,57.899408


In [ ]:
new_df["age_in_years"] = new_df["age_in_years"].astype("int32")

In [ ]:
print("age ranges from {} to {}, with a mean of {} and median of {} and standard deviation of {}".format(
    new_df.age_in_years.min(),  new_df.age_in_years.max(),  
    new_df.age_in_years.mean(),new_df.age_in_years.median(),
    new_df.age_in_years.std()
)
)

age ranges from 20 to 91, with a mean of 55.27744680851064 and median of 55.0 and standard deviation of 11.265775677601631


In [ ]:

x = new_df.age_in_years
n, bins, patches = plt.hist(x, 10, facecolor='b')
plt.xlabel('Age in years')
plt.ylabel('Frequency')
plt.title('Age distribution of Mirai Input (all patients)')
# plt.text(images.image_means.mean(), images.image_means.std(), r'$\mu=,\ \sigma=15$')
plt.xlim(new_df.age_in_years.min() - 10 , new_df.age_in_years.max() + 10)
#plt.ylim(0, 0.03)
plt.grid(True)
plt.show()

In [ ]:
under_50 = new_df[new_df.age_in_years < 50]
middle = new_df[(new_df.age_in_years >= 50) & (new_df.age_in_years<75) ]
over = new_df[new_df.age_in_years >= 75]

In [ ]:
keep = ['patient_id', 'exam_id', 'laterality', 'view', 'years_to_cancer', 'years_to_last_followup', 'file_path', 'split_group']
under_50= under_50[keep]
middle = middle[keep]
over = over[keep]

In [ ]:
under_50.to_csv(path_or_buf= "../data/cleaned/mirai_under_50.csv")
middle.to_csv(path_or_buf= "../data/cleaned/mirai_age51_74.csv")
over.to_csv(path_or_buf= "../data/cleaned/mirai_over74.csv")

In [ ]:
#case and control & exam distribution for age groups
au50 = new_df[new_df.age_in_years < 50]
aMid = new_df[(new_df.age_in_years >= 50) & (new_df.age_in_years <7 5) ]
aOver = new_df[new_df.age_in_years >= 75]

In [ ]:
print("in the under-50 group, there are {} case exams for {} unique women, \m {} control exams for {} unique women".format(
    au50[au50.years_to_cancer!=100].exam_id.nunique(), au50[au50.years_to_cancer!=100].study_id.nunique(),
    au50[au50.years_to_cancer==100].exam_id.nunique(), au50[au50.years_to_cancer==100].study_id.nunique()
))

in the under-50 group, there are 90 case exams for 41 unique women, \m 2066 control exams for 752 unique women


In [ ]:
print("in the 50 to <75 years group, there are {} case exams for {} unique women, \m {} control exams for {} unique women".format(
    aMid[aMid.years_to_cancer!=100].exam_id.nunique(), aMid[aMid.years_to_cancer!=100].study_id.nunique(),
    aMid[aMid.years_to_cancer==100].exam_id.nunique(), aMid[aMid.years_to_cancer==100].study_id.nunique()
))

in the 50 to <75 years group, there are 191 case exams for 83 unique women, \m 3606 control exams for 946 unique women


In [ ]:
print("in the 75 years and older group, there are {} case exams for {} unique women, \m {} control exams for {} unique women".format(
    aOver[aOver.years_to_cancer!=100].exam_id.nunique(), aOver[aOver.years_to_cancer!=100].study_id.nunique(),
    aOver[aOver.years_to_cancer==100].exam_id.nunique(), aOver[aOver.years_to_cancer==100].study_id.nunique()
))

in the 75 years and older group, there are 35 case exams for 18 unique women, \m 274 control exams for 97 unique women


In [ ]:
print("in the full input cohort, there are {} case exams for {} unique women, \m {} control exams for {} unique women".format(
    new_df[new_df.years_to_cancer!=100].exam_id.nunique(), new_df[new_df.years_to_cancer!=100].study_id.nunique(),
    new_df[new_df.years_to_cancer==100].exam_id.nunique(), new_df[new_df.years_to_cancer==100].study_id.nunique()
))

in the full input cohort, there are 316 case exams for 130 unique women, \m 5946 control exams for 1573 unique women


In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
results_path= '/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/cleaned/interim_ddfull_cohort_output.csv'
input_path = '/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/cleaned/deduplicated_all.csv'